Name: Ziwei Liu
Student ID: 3766789

In [12]:
import os
import zipfile
import torch

import requests
from torch.utils.data import DataLoader
from torchvision import transforms as T
from torchvision.datasets import CIFAR10
from tqdm import tqdm
import torch.nn as nn

In [13]:
from examples.models.densenet import densenet121

In [14]:
threads = 16
torch.set_num_threads(threads)

#maybe better performance (Jupyter)
#%env OMP_PLACES=cores
#%env OMP_PROC_BIND=close
#%env OMP_WAIT_POLICY=active

#WSL
os.environ['OMP_PLACES'] = 'cores'
os.environ['OMP_PROC_BIND'] = 'close'
os.environ['OMP_WAIT_POLICY'] = 'active'

In [15]:
#axx_mult = 'mul8s_acc'

In [16]:
axx_mult = 'exact'
model = densenet121(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation

Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exact, skipping build step...
Loading extension module PyInit_conv2d_exact...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exact, skipping build step...
Loading extension module PyInit_conv2d_exact...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exact, skipping build step...
Loading extension module PyInit_conv2d_exact...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exact, skipping build step...
Loading extension module PyInit_conv2d_exact...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_exa

DenseNet(
  (features): Sequential(
    (conv0): AdaPT_Conv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
      (quantizer_w): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
    )
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): AdaPT_Conv2d(
          64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False
          (quantizer): TensorQuantizer(8bit per-tensor amax=dynamic calibrator=HistogramCalibrator quant)
          (quantizer_w): TensorQuant

In [19]:
def val_dataloader(mean = (0.4914, 0.4822, 0.4465), std = (0.2471, 0.2435, 0.2616)):

    transform = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean, std),
        ]
    )
    dataset = CIFAR10(root="datasets/cifar10_data", train=False, download=True, transform=transform)
    dataloader = DataLoader(
        dataset,
        batch_size=1024,
        num_workers=0,
        drop_last=True,
        pin_memory=False,
    )
    return dataloader

transform = T.Compose(
        [
            T.RandomCrop(32, padding=4),
            T.RandomHorizontalFlip(),
            T.ToTensor(),
            T.Normalize(mean = (0.4914, 0.4822, 0.4465), std = (0.2471, 0.2435, 0.2616)),
        ]
    )
dataset = CIFAR10(root="datasets/cifar10_data", train=True, download=True, transform=transform)

evens = list(range(0, len(dataset), 10))
trainset_1 = torch.utils.data.Subset(dataset, evens)

data = val_dataloader()

# data_t is used for calibration purposes and is a subset of train-set
data_t = DataLoader(trainset_1, batch_size=128,
                                            shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [20]:
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import calib

def collect_stats(model, data_loader, num_batches):
     """Feed data to the network and collect statistic"""

     # Enable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.disable_quant()
                 module.enable_calib()
             else:
                 module.disable()

     for i, (image, _) in tqdm(enumerate(data_loader), total=num_batches):
         model(image.cpu())
         if i >= num_batches:
             break

     # Disable calibrators
     for name, module in model.named_modules():
         if isinstance(module, quant_nn.TensorQuantizer):
             if module._calibrator is not None:
                 module.enable_quant()
                 module.disable_calib()
             else:
                 module.enable()

def compute_amax(model, **kwargs):
 # Load calib result
 for name, module in model.named_modules():
     if isinstance(module, quant_nn.TensorQuantizer):
         if module._calibrator is not None:
             if isinstance(module._calibrator, calib.MaxCalibrator):
                 module.load_calib_amax()
             else:
                 module.load_calib_amax(**kwargs)
         print(F"{name:40}: {module}")
 model.cpu()

# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)

100%|██████████| 2/2 [00:09<00:00,  4.90s/it]
W0222 02:23:02.289330 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:23:02.289930 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:23:02.290439 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:23:02.290885 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:23:02.291328 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:23:02.291781 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:23:02.292217 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:23:02.292624 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:23:02.293027 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:23:02.293437 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:23:02.293871 140182088872128 tensor

features.conv0.quantizer                : TensorQuantizer(8bit per-tensor amax=2.1255 calibrator=HistogramCalibrator quant)
features.conv0.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.1544 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv1.quantizer: TensorQuantizer(8bit per-tensor amax=0.4862 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv1.quantizer_w: TensorQuantizer(8bit per-tensor amax=0.0649 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv2.quantizer: TensorQuantizer(8bit per-tensor amax=0.1260 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv2.quantizer_w: TensorQuantizer(8bit per-tensor amax=0.0350 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer2.conv1.quantizer: TensorQuantizer(8bit per-tensor amax=0.3875 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer2.conv1.quantizer_w: TensorQuantizer(8bit per-tensor amax

In [21]:
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

100%|██████████| 9/9 [01:52<00:00, 12.47s/it]

112.20802609995008
Accuracy of the network on the 10000 test images: 93.8043 %


In [22]:
axx_mult = 'appro1'
model = densenet121(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation
# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

# Retraining for 15 epochs
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(15):
    print(f"Epoch {epoch + 1}/15")
    train_one_epoch(model, criterion, optimizer, data_t, "cpu", epoch, 1)
    lr_scheduler.step()

# Inference after retraining
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro1, skipping build step...
Loading extension module PyInit_conv2d_appro1...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro1, skipping build step...
Loading extension module PyInit_conv2d_appro1...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro1, skipping build step...
Loading extension module PyInit_conv2d_appro1...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro1, skipping build step...
Loading extension module PyInit_conv2d_appro1...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_co

100%|██████████| 2/2 [00:06<00:00,  3.05s/it]
W0222 02:25:04.695062 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:25:04.695796 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:25:04.696300 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:25:04.696727 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:25:04.697151 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:25:04.697560 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:25:04.697994 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:25:04.698407 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:25:04.698829 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:25:04.699223 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 02:25:04.699649 140182088872128 tensor

features.conv0.quantizer                : TensorQuantizer(8bit per-tensor amax=2.1255 calibrator=HistogramCalibrator quant)
features.conv0.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.1544 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv1.quantizer: TensorQuantizer(8bit per-tensor amax=0.5013 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv1.quantizer_w: TensorQuantizer(8bit per-tensor amax=0.0649 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv2.quantizer: TensorQuantizer(8bit per-tensor amax=0.1359 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv2.quantizer_w: TensorQuantizer(8bit per-tensor amax=0.0350 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer2.conv1.quantizer: TensorQuantizer(8bit per-tensor amax=0.4326 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer2.conv1.quantizer_w: TensorQuantizer(8bit per-tensor amax

100%|██████████| 9/9 [01:51<00:00, 12.43s/it]


111.849509883672
Accuracy of the network on the 10000 test images: 11.4041 %
Epoch 1/15
Epoch: [0]  [ 0/40]  eta: 0:07:46  lr: 0.0001  img/s: 10.991093558620445  loss: 2.1893 (2.1893)  acc1: 24.2188 (24.2188)  acc5: 75.0000 (75.0000)  time: 11.6640  data: 0.0182
Epoch: [0]  [ 1/40]  eta: 0:07:37  lr: 0.0001  img/s: 10.853583712329716  loss: 2.1668 (2.1780)  acc1: 24.2188 (25.0000)  acc5: 75.0000 (75.3906)  time: 11.7375  data: 0.0179
Epoch: [0]  [ 2/40]  eta: 0:07:26  lr: 0.0001  img/s: 10.892789282781235  loss: 2.1668 (2.1420)  acc1: 25.7812 (29.1667)  acc5: 75.7812 (79.1667)  time: 11.7479  data: 0.0178
Epoch: [0]  [ 3/40]  eta: 0:07:12  lr: 0.0001  img/s: 11.146370781514209  loss: 2.0698 (2.0950)  acc1: 25.7812 (32.2266)  acc5: 75.7812 (81.0547)  time: 11.6862  data: 0.0178
Epoch: [0]  [ 4/40]  eta: 0:07:00  lr: 0.0001  img/s: 11.018980067023822  loss: 2.0698 (2.0169)  acc1: 37.5000 (37.5000)  acc5: 86.7188 (83.4375)  time: 11.6757  data: 0.0177
Epoch: [0]  [ 5/40]  eta: 0:06:47  lr

100%|██████████| 9/9 [01:53<00:00, 12.61s/it]

113.51266868971288
Accuracy of the network on the 10000 test images: 84.7656 %


In [23]:
axx_mult = 'appro2'
model = densenet121(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation
# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

# Retraining for 15 epochs
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(15):
    print(f"Epoch {epoch + 1}/15")
    train_one_epoch(model, criterion, optimizer, data_t, "cpu", epoch, 1)
    lr_scheduler.step()

# Inference after retraining
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

Using /root/.cache/torch_extensions as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/PyInit_conv2d_appro2...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_appro2/build.ninja...
Building extension module PyInit_conv2d_appro2...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
[1/2] c++ -MMD -MF axx_conv2d.o.d -DTORCH_EXTENSION_NAME=PyInit_conv2d_appro2 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /root/miniconda3/lib/python3.8/site-packages/torch/include -isystem /root/miniconda3/lib/python3.8/site-packages/torch/include/torch/csrc/api/include -isystem /root/miniconda3/lib/python3.8/site-packages/torch/include/TH -isystem /root/miniconda3/lib/python3.8/site-packages/torch/include/THC -isystem /root/miniconda3/include/python3.8 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -s

100%|██████████| 2/2 [00:05<00:00,  3.00s/it]
W0222 04:22:57.087216 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 04:22:57.087850 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 04:22:57.088297 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 04:22:57.088697 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 04:22:57.089094 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 04:22:57.089447 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 04:22:57.089828 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 04:22:57.090205 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 04:22:57.090579 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 04:22:57.090940 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 04:22:57.091270 140182088872128 tensor

features.conv0.quantizer                : TensorQuantizer(8bit per-tensor amax=2.1255 calibrator=HistogramCalibrator quant)
features.conv0.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.1544 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv1.quantizer: TensorQuantizer(8bit per-tensor amax=0.5031 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv1.quantizer_w: TensorQuantizer(8bit per-tensor amax=0.0649 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv2.quantizer: TensorQuantizer(8bit per-tensor amax=0.1361 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv2.quantizer_w: TensorQuantizer(8bit per-tensor amax=0.0350 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer2.conv1.quantizer: TensorQuantizer(8bit per-tensor amax=0.4313 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer2.conv1.quantizer_w: TensorQuantizer(8bit per-tensor amax

100%|██████████| 9/9 [01:53<00:00, 12.67s/it]


113.99896082095802
Accuracy of the network on the 10000 test images: 10.1128 %
Epoch 1/15
Epoch: [0]  [ 0/40]  eta: 0:07:50  lr: 0.0001  img/s: 10.90302250433474  loss: 2.3043 (2.3043)  acc1: 10.9375 (10.9375)  acc5: 50.7812 (50.7812)  time: 11.7582  data: 0.0184
Epoch: [0]  [ 1/40]  eta: 0:07:40  lr: 0.0001  img/s: 10.814878073823268  loss: 2.2886 (2.2965)  acc1: 6.2500 (8.5938)  acc5: 50.7812 (57.4219)  time: 11.8060  data: 0.0183
Epoch: [0]  [ 2/40]  eta: 0:07:28  lr: 0.0001  img/s: 10.90200449359933  loss: 2.3043 (2.3104)  acc1: 10.9375 (9.8958)  acc5: 50.7812 (52.8646)  time: 11.7903  data: 0.0182
Epoch: [0]  [ 3/40]  eta: 0:07:16  lr: 0.0001  img/s: 10.873744109702397  loss: 2.3043 (2.3119)  acc1: 8.5938 (9.5703)  acc5: 45.3125 (50.9766)  time: 11.7901  data: 0.0181
Epoch: [0]  [ 4/40]  eta: 0:07:05  lr: 0.0001  img/s: 10.753213420128558  loss: 2.3043 (2.3087)  acc1: 10.9375 (10.0000)  acc5: 50.7812 (51.8750)  time: 11.8163  data: 0.0181
Epoch: [0]  [ 5/40]  eta: 0:06:53  lr: 0.0

100%|██████████| 9/9 [01:49<00:00, 12.19s/it]

109.72209514304996
Accuracy of the network on the 10000 test images: 10.3190 %


In [24]:
axx_mult = 'appro3'
model = densenet121(pretrained=True, axx_mult = axx_mult)
model.eval() # for evaluation
# It is a bit slow since we collect histograms on CPU
with torch.no_grad():
    stats = collect_stats(model, data_t, num_batches=2)
    amax = compute_amax(model, method="percentile", percentile=99.99)
    
# Inference without retraining
import timeit
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

# Retraining for 15 epochs
from adapt.references.classification.train import evaluate, train_one_epoch, load_data

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

for epoch in range(15):
    print(f"Epoch {epoch + 1}/15")
    train_one_epoch(model, criterion, optimizer, data_t, "cpu", epoch, 1)
    lr_scheduler.step()

# Inference after retraining
correct = 0
total = 0

model.eval()
start_time = timeit.default_timer()
with torch.no_grad():
    for iteraction, (images, labels) in tqdm(enumerate(data), total=len(data)):
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(timeit.default_timer() - start_time)
print('Accuracy of the network on the 10000 test images: %.4f %%' % (
    100 * correct / total))

Using /root/.cache/torch_extensions as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/PyInit_conv2d_appro3/build.ninja...
Building extension module PyInit_conv2d_appro3...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module PyInit_conv2d_appro3...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro3, skipping build step...
Loading extension module PyInit_conv2d_appro3...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro3, skipping build step...
Loading extension module PyInit_conv2d_appro3...
Using /root/.cache/torch_extensions as PyTorch extensions root...
No modifications detected for re-loaded extension module PyInit_conv2d_appro3, skipping build step...
Loadi

100%|██████████| 2/2 [00:06<00:00,  3.01s/it]
W0222 06:20:35.301978 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 06:20:35.302585 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 06:20:35.303064 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 06:20:35.303492 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 06:20:35.303942 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 06:20:35.304349 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 06:20:35.304764 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 06:20:35.306354 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 06:20:35.306749 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 06:20:35.307162 140182088872128 tensor_quantizer.py:174] Disable HistogramCalibrator
W0222 06:20:35.307581 140182088872128 tensor

features.conv0.quantizer                : TensorQuantizer(8bit per-tensor amax=2.1255 calibrator=HistogramCalibrator quant)
features.conv0.quantizer_w              : TensorQuantizer(8bit per-tensor amax=0.1544 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv1.quantizer: TensorQuantizer(8bit per-tensor amax=0.5012 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv1.quantizer_w: TensorQuantizer(8bit per-tensor amax=0.0649 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv2.quantizer: TensorQuantizer(8bit per-tensor amax=0.1368 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer1.conv2.quantizer_w: TensorQuantizer(8bit per-tensor amax=0.0350 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer2.conv1.quantizer: TensorQuantizer(8bit per-tensor amax=0.4310 calibrator=HistogramCalibrator quant)
features.denseblock1.denselayer2.conv1.quantizer_w: TensorQuantizer(8bit per-tensor amax

100%|██████████| 9/9 [01:54<00:00, 12.71s/it]


114.43707771226764
Accuracy of the network on the 10000 test images: 10.1128 %
Epoch 1/15
Epoch: [0]  [ 0/40]  eta: 0:07:47  lr: 0.0001  img/s: 10.97944989219044  loss: 2.2749 (2.2749)  acc1: 14.8438 (14.8438)  acc5: 57.0312 (57.0312)  time: 11.6757  data: 0.0175
Epoch: [0]  [ 1/40]  eta: 0:07:39  lr: 0.0001  img/s: 10.763719819928788  loss: 2.2550 (2.2649)  acc1: 14.8438 (17.1875)  acc5: 57.0312 (57.8125)  time: 11.7928  data: 0.0178
Epoch: [0]  [ 2/40]  eta: 0:07:27  lr: 0.0001  img/s: 10.93957636013871  loss: 2.2550 (2.2484)  acc1: 19.5312 (18.2292)  acc5: 58.5938 (62.2396)  time: 11.7680  data: 0.0178
Epoch: [0]  [ 3/40]  eta: 0:07:14  lr: 0.0001  img/s: 11.013171309571273  loss: 2.2154 (2.2314)  acc1: 19.5312 (19.3359)  acc5: 58.5938 (65.0391)  time: 11.7360  data: 0.0177
Epoch: [0]  [ 4/40]  eta: 0:07:03  lr: 0.0001  img/s: 10.852432975823758  loss: 2.2154 (2.2058)  acc1: 20.3125 (23.1250)  acc5: 71.0938 (69.0625)  time: 11.7513  data: 0.0177
Epoch: [0]  [ 5/40]  eta: 0:06:51  lr

100%|██████████| 9/9 [01:52<00:00, 12.51s/it]

112.58940046280622
Accuracy of the network on the 10000 test images: 75.0760 %
